# Ejercicio 1

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F

## Task 1

In [2]:
iris = load_iris()

X = iris.data
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Task 2

In [3]:
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleFeedforwardNN, self).__init__()
        
        #layers
        self.input_layer = nn.Linear(input_size, hidden_size1)
        self.hidden_layer1 = nn.Linear(hidden_size1, hidden_size2)
        self.hidden_layer2 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        #relu activation function for input layer
        x = F.relu(self.input_layer(x))
        
        #relu activation function for hidden layer 1
        x = F.relu(self.hidden_layer1(x))
        
        #log softmax activation function for hidden layer 2
        x = F.log_softmax(self.hidden_layer2(x), dim=1)
        
        return x

In [5]:
input_size = 4
hidden_size1 = 10
hidden_size2 = 8
output_size = 3

model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)

# Ejercicio 2